# Worksheet 5 — Linear Regression from Scratch

**Dataset:** `student.csv`

## Objective
To predict the marks obtained in **Writing** based on the marks of **Math** and **Reading**.

## Submission Instructions
- Submit a single notebook containing:
  1. Clean and well-documented code.
  2. Outputs and visualizations.
  3. Detailed explanations and analysis for all steps.
- Ensure all cells are executed before submission.

---

## Key Assumptions
- **No bias/intercept term**: We assume bias = 0 for simplicity.
- The model is: $Y = W^T X$

Where:
- $W \in \mathbb{R}^d$ (weight vector)
- $X \in \mathbb{R}^{d \times n}$ (feature matrix)
- $Y \in \mathbb{R}^n$ (target vector)

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple

# Configure display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)
np.set_printoptions(precision=4, suppress=True)

%matplotlib inline
sns.set_style('whitegrid')

## Step 1 — Data Understanding, Analysis and Preparation

### To-Do 1: Load and Explore Dataset

In [ ]:
# 1. Read the dataset
DATA_PATH = 'student.csv'

try:
    data = pd.read_csv(DATA_PATH)
    print(f"✓ Dataset loaded successfully!")
    print(f"Shape: {data.shape}")
except FileNotFoundError as e:
    raise FileNotFoundError(
        f"Could not find {DATA_PATH!r}. Please place student.csv in the same folder as this notebook."
    ) from e

# 2. Display top 5 rows
print("\n" + "="*80)
print("TOP 5 ROWS:")
print("="*80)
display(data.head())

# 3. Display bottom 5 rows
print("\n" + "="*80)
print("BOTTOM 5 ROWS:")
print("="*80)
display(data.tail())

In [ ]:
# 3. Print dataset information
print("="*80)
print("DATASET INFORMATION:")
print("="*80)
data.info()

print("\n" + "="*80)
print("MISSING VALUES CHECK:")
print("="*80)
print(data.isnull().sum())

print("\n" + "="*80)
print("DATA TYPES:")
print("="*80)
print(data.dtypes)

In [ ]:
# 4. Descriptive statistics
print("="*80)
print("DESCRIPTIVE STATISTICS:")
print("="*80)
display(data.describe())

# Additional statistics
print("\n" + "="*80)
print("CORRELATION MATRIX:")
print("="*80)
numeric_cols = data.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 0:
    display(data[numeric_cols].corr())
    
    # Visualize correlations
    plt.figure(figsize=(8, 6))
    sns.heatmap(data[numeric_cols].corr(), annot=True, fmt='.3f', cmap='coolwarm', center=0)
    plt.title('Correlation Heatmap')
    plt.tight_layout()
    plt.show()

In [ ]:
# 5. Split data into Features (X) and Target (Y)
# Assuming the dataset has columns: Math, Reading, Writing
# We want to predict Writing based on Math and Reading

# Check if expected columns exist
expected_cols = ['Math', 'Reading', 'Writing']
if all(col in data.columns for col in expected_cols):
    X_full = data[['Math', 'Reading']].values  # Features
    Y_full = data['Writing'].values  # Target
    print("✓ Features (X) and Target (Y) successfully separated!")
    print(f"  X shape: {X_full.shape} (samples × features)")
    print(f"  Y shape: {Y_full.shape} (samples)")
else:
    print(f"Warning: Expected columns {expected_cols} not found.")
    print(f"Available columns: {list(data.columns)}")
    # Fallback: use all numeric columns except the last as features
    numeric_data = data.select_dtypes(include=[np.number])
    X_full = numeric_data.iloc[:, :-1].values
    Y_full = numeric_data.iloc[:, -1].values
    print(f"Using fallback: X shape {X_full.shape}, Y shape {Y_full.shape}")

### To-Do 2: Create Matrices

We define:
- **Weight matrix** $W \in \mathbb{R}^d$ where $d$ is the number of features
- **Feature matrix** $X \in \mathbb{R}^{d \times n}$ where $n$ is the number of samples
- **Target vector** $Y \in \mathbb{R}^n$

**Note:** No bias column (column of 1s) is added since we assume bias = 0.

In [ ]:
# Matrix dimensions
n_samples, n_features = X_full.shape
print(f"Number of samples (n): {n_samples}")
print(f"Number of features (d): {n_features}")
print(f"\nMatrix shapes:")
print(f"  X: {X_full.shape} (n_samples × n_features)")
print(f"  Y: {Y_full.shape} (n_samples,)")
print(f"  W will be: ({n_features},) after initialization")

### To-Do 3: Train-Test Split (from scratch)

Split the dataset into training (70% or 80%) and test (30% or 20%) sets.

In [ ]:
def train_test_split(X, Y, test_size=0.2, random_state=42):
    """
    Split data into training and test sets from scratch.
    
    Parameters:
        X: Feature matrix (n_samples, n_features)
        Y: Target vector (n_samples,)
        test_size: Proportion of data for testing (default 0.2 = 20%)
        random_state: Random seed for reproducibility
    
    Returns:
        X_train, X_test, Y_train, Y_test
    """
    np.random.seed(random_state)
    n_samples = X.shape[0]
    
    # Create shuffled indices
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    # Calculate split point
    test_samples = int(n_samples * test_size)
    train_samples = n_samples - test_samples
    
    # Split indices
    train_indices = indices[:train_samples]
    test_indices = indices[train_samples:]
    
    # Split data
    X_train = X[train_indices]
    X_test = X[test_indices]
    Y_train = Y[train_indices]
    Y_test = Y[test_indices]
    
    return X_train, X_test, Y_train, Y_test

# Perform the split (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X_full, Y_full, test_size=0.2, random_state=42)

print("="*80)
print("TRAIN-TEST SPLIT COMPLETED:")
print("="*80)
print(f"Training set: X_train={X_train.shape}, Y_train={Y_train.shape}")
print(f"Test set:     X_test={X_test.shape}, Y_test={Y_test.shape}")
print(f"\nSplit ratio: {100*(1-0.2):.0f}% train, {100*0.2:.0f}% test")

## Step 2 — Build a Cost Function

The cost function for regression is the **Mean Squared Error (MSE)**:

$$L(w) = \frac{1}{2n} \sum_{i=1}^{n} (y_{pred}^{(i)} - y_i)^2$$

where $y_{pred}(w) = W^T X$

### To-Do 4: Implement Cost Function

In [ ]:
def cost_function(X, Y, W):
    """
    Calculate the Mean Squared Error (MSE) cost function.
    
    Parameters:
        X: Feature matrix (n_samples, n_features)
        Y: Target vector (n_samples,)
        W: Weight vector (n_features,)
    
    Returns:
        cost: Mean squared error
    """
    n = len(Y)  # Number of samples
    
    # Compute predictions: Y_pred = X @ W
    Y_pred = np.dot(X, W)
    
    # Calculate squared errors
    squared_errors = (Y_pred - Y) ** 2
    
    # Mean squared error (divided by 2n as per formula)
    cost = np.sum(squared_errors) / (2 * n)
    
    return cost

print("✓ Cost function implemented successfully!")

### To-Do 5: Test the Cost Function

**Test Case:**
Given:
- $X = \begin{bmatrix} 1 & 3 & 5 \\ 2 & 4 & 6 \end{bmatrix}^T$ 
- $Y = \begin{bmatrix} 3 & 7 & 11 \end{bmatrix}^T$
- $W = \begin{bmatrix} 1 \\ 1 \end{bmatrix}$

Expected: **cost = 0**

In [ ]:
# Test case for cost function
X_test = np.array([[1, 2], [3, 4], [5, 6]])
Y_test = np.array([3, 7, 11])
W_test = np.array([1, 1])

cost = cost_function(X_test, Y_test, W_test)

print("="*80)
print("COST FUNCTION TEST:")
print("="*80)
print(f"Test input:")
print(f"  X_test = \n{X_test}")
print(f"  Y_test = {Y_test}")
print(f"  W_test = {W_test}")
print(f"\nPredictions: {np.dot(X_test, W_test)}")
print(f"Cost function output: {cost}")

if cost == 0:
    print("\n✓ TEST PASSED! Proceed further.")
else:
    print(f"\n✗ TEST FAILED! Expected 0, got {cost}")
    print("Something went wrong: Reimplement the cost function")

## Step 3 — Gradient Descent for Linear Regression

**Objective:** Learn the parameters $W$ (weights) assuming bias $b = 0$.

### Key Formulas

**Hypothesis:**
$$h_w(x) = W^T x$$

**Loss Function:**
$$\text{Loss} = (h_w(x) - y)^2$$

**Gradient:**
$$\frac{\partial \text{Loss}}{\partial w} = 2 \cdot (h_w(x) - y) \cdot x$$

**Update Rule:**
$$W^{(j+1)} = W^{(j)} - \alpha \cdot \frac{1}{m} \sum_{i=1}^{m} (h_w(x^{(i)}) - y^{(i)}) \cdot x^{(i)}$$

where:
- $\alpha$ = learning rate
- $m$ = number of training samples

### To-Do 6: Implement Gradient Descent

In [ ]:
def gradient_descent(X, Y, W, alpha, iterations):
    """
    Perform gradient descent to optimize the parameters of a linear regression model.
    
    Parameters:
        X (numpy.ndarray): Feature matrix (m x n).
        Y (numpy.ndarray): Target vector (m,).
        W (numpy.ndarray): Initial guess for parameters (n,).
        alpha (float): Learning rate.
        iterations (int): Number of iterations for gradient descent.
    
    Returns:
        tuple: A tuple containing:
            - W_update (numpy.ndarray): Updated parameters (n,).
            - cost_history (list): History of cost values over iterations.
    """
    # Initialize cost history
    cost_history = [0] * iterations
    
    # Number of samples
    m = len(Y)
    
    # Make a copy of W to avoid modifying the original
    W_update = W.copy()
    
    for iteration in range(iterations):
        # Step 1: Hypothesis Values (Predictions)
        Y_pred = np.dot(X, W_update)
        
        # Step 2: Difference between Hypothesis and Actual Y (Loss/Error)
        loss = Y_pred - Y
        
        # Step 3: Gradient Calculation
        # dw = (1/m) * X^T * (Y_pred - Y)
        dw = (1/m) * np.dot(X.T, loss)
        
        # Step 4: Updating Values of W using Gradient
        W_update = W_update - alpha * dw
        
        # Step 5: New Cost Value
        cost = cost_function(X, Y, W_update)
        cost_history[iteration] = cost
    
    return W_update, cost_history

print("✓ Gradient descent function implemented successfully!")

### To-Do 7: Test Gradient Descent

In [ ]:
# Generate random test data
np.random.seed(0)  # For reproducibility
X_gd_test = np.random.rand(100, 3)  # 100 samples, 3 features
Y_gd_test = np.random.rand(100)
W_gd_test = np.random.rand(3)  # Initial guess for parameters

# Set hyperparameters
alpha = 0.01
iterations = 1000

# Test the gradient_descent function
print("="*80)
print("GRADIENT DESCENT TEST:")
print("="*80)
print(f"Training on random data: {X_gd_test.shape[0]} samples, {X_gd_test.shape[1]} features")
print(f"Learning rate (alpha): {alpha}")
print(f"Iterations: {iterations}")
print(f"\nInitial weights: {W_gd_test}")
print(f"Initial cost: {cost_function(X_gd_test, Y_gd_test, W_gd_test):.6f}")

final_params, cost_history = gradient_descent(X_gd_test, Y_gd_test, W_gd_test, alpha, iterations)

# Print the final parameters and cost history
print(f"\n✓ Gradient descent completed!")
print(f"Final Parameters: {final_params}")
print(f"Final cost: {cost_history[-1]:.6f}")
print(f"\nCost History (first 10 iterations): {[f'{c:.6f}' for c in cost_history[:10]]}")
print(f"Cost History (last 10 iterations): {[f'{c:.6f}' for c in cost_history[-10:]]}")

# Visualize cost over iterations
plt.figure(figsize=(10, 5))
plt.plot(cost_history, linewidth=2)
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.title('Cost Function vs Iterations (Gradient Descent Test)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Step 4 — Evaluate the Model

### To-Do 8: Implement RMSE (Root Mean Squared Error)

$$\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$$

In [ ]:
def rmse(Y, Y_pred):
    """
    Calculate the Root Mean Squared Error.
    
    Input Arguments:
        Y: Array of actual (target) dependent variables.
        Y_pred: Array of predicted dependent variables.
    
    Output Arguments:
        rmse: Root Mean Square Error.
    """
    # Calculate squared differences
    squared_diff = (Y - Y_pred) ** 2
    
    # Mean of squared differences
    mean_squared_error = np.mean(squared_diff)
    
    # Square root
    rmse = np.sqrt(mean_squared_error)
    
    return rmse

print("✓ RMSE function implemented successfully!")

### To-Do 9: Implement R² (Coefficient of Determination)

$$R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$$

where:
- $SS_{res} = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$ (Sum of Squared Residuals)
- $SS_{tot} = \sum_{i=1}^{n} (y_i - \bar{y})^2$ (Total Sum of Squares)
- $\bar{y}$ = mean of actual values

In [ ]:
def r2(Y, Y_pred):
    """
    Calculate the R-Squared (Coefficient of Determination).
    
    Input Arguments:
        Y: Array of actual (target) dependent variables.
        Y_pred: Array of predicted dependent variables.
    
    Output Arguments:
        r2: R-Squared Error.
    """
    # Calculate mean of actual values
    mean_y = np.mean(Y)
    
    # Total Sum of Squares (SS_tot)
    ss_tot = np.sum((Y - mean_y) ** 2)
    
    # Residual Sum of Squares (SS_res)
    ss_res = np.sum((Y - Y_pred) ** 2)
    
    # R-squared
    r2 = 1 - (ss_res / ss_tot)
    
    return r2

print("✓ R² function implemented successfully!")

## Step 5 — Main Function to Integrate All Steps

### To-Do 10: Create Main Integration Function

This function brings everything together:
1. Load and prepare data
2. Train-test split
3. Initialize weights and hyperparameters
4. Run gradient descent
5. Make predictions
6. Evaluate using RMSE and R²

In [ ]:
def main():
    """
    Main function to integrate all steps:
    - Load data
    - Split into train/test
    - Run gradient descent
    - Evaluate model
    """
    print("="*80)
    print("MAIN EXECUTION: LINEAR REGRESSION FROM SCRATCH")
    print("="*80)
    
    # Step 1: Load the dataset
    data = pd.read_csv('student.csv')
    print(f"\n✓ Dataset loaded: {data.shape[0]} samples, {data.shape[1]} columns")
    
    # Step 2: Split the data into features (X) and target (Y)
    X = data[['Math', 'Reading']].values  # Features: Math and Reading marks
    Y = data['Writing'].values  # Target: Writing marks
    print(f"✓ Features: Math and Reading → Target: Writing")
    
    # Step 3: Split the data into training and test sets (80% train, 20% test)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    print(f"✓ Train-test split: {X_train.shape[0]} train, {X_test.shape[0]} test")
    
    # Step 4: Initialize weights (W) to zeros, learning rate and number of iterations
    W = np.zeros(X_train.shape[1])  # Initialize weights
    alpha = 0.00001  # Learning rate
    iterations = 1000  # Number of iterations for gradient descent
    print(f"✓ Hyperparameters: alpha={alpha}, iterations={iterations}")
    print(f"✓ Initial weights: {W}")
    
    # Step 5: Perform Gradient Descent
    print(f"\n{'='*80}")
    print("TRAINING MODEL WITH GRADIENT DESCENT...")
    print(f"{'='*80}")
    W_optimal, cost_history = gradient_descent(X_train, Y_train, W, alpha, iterations)
    print(f"✓ Training complete!")
    print(f"  Initial cost: {cost_history[0]:.4f}")
    print(f"  Final cost: {cost_history[-1]:.4f}")
    print(f"  Cost reduction: {cost_history[0] - cost_history[-1]:.4f}")
    
    # Step 6: Make predictions on the test set
    Y_pred = np.dot(X_test, W_optimal)
    print(f"\n✓ Predictions made on test set")
    
    # Step 7: Evaluate the model using RMSE and R-Squared
    model_rmse = rmse(Y_test, Y_pred)
    model_r2 = r2(Y_test, Y_pred)
    
    # Step 8: Output the results
    print(f"\n{'='*80}")
    print("FINAL RESULTS:")
    print(f"{'='*80}")
    print(f"Final Weights: {W_optimal}")
    print(f"Cost History (First 10 iterations): {cost_history[:10]}")
    print(f"RMSE on Test Set: {model_rmse:.4f}")
    print(f"R-Squared on Test Set: {model_r2:.4f}")
    print(f"{'='*80}")
    
    # Visualization 1: Cost over iterations
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(cost_history, linewidth=2, color='blue')
    plt.xlabel('Iteration')
    plt.ylabel('Cost')
    plt.title('Cost Function vs Iterations')
    plt.grid(True, alpha=0.3)
    
    # Visualization 2: Actual vs Predicted
    plt.subplot(1, 2, 2)
    plt.scatter(Y_test, Y_pred, alpha=0.6, edgecolors='k')
    plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r--', lw=2, label='Perfect prediction')
    plt.xlabel('Actual Writing Score')
    plt.ylabel('Predicted Writing Score')
    plt.title('Actual vs Predicted')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return W_optimal, cost_history, model_rmse, model_r2

# Execute the main function
if __name__ == "__main__":
    W_optimal, cost_history, model_rmse, model_r2 = main()

## To-Do 11 — Present Your Findings & Experimentation

### Model Performance Analysis

Let's analyze whether our model is **overfitting**, **underfitting**, or has **acceptable performance**:

**Interpretation Guidelines:**
- **R² Score:**
  - Close to 1.0 → Excellent fit
  - 0.7 - 0.9 → Good fit
  - 0.5 - 0.7 → Moderate fit
  - < 0.5 → Poor fit
  
- **RMSE:** 
  - Lower is better
  - Compare to the range of the target variable
  
- **Overfitting indicators:**
  - Very high training performance but poor test performance
  - Large gap between train and test metrics
  
- **Underfitting indicators:**
  - Poor performance on both training and test sets
  - Model is too simple to capture the underlying pattern

In [ ]:
# Evaluate on training set as well to compare
Y_train_pred = np.dot(X_train, W_optimal)
train_rmse = rmse(Y_train, Y_train_pred)
train_r2 = r2(Y_train, Y_train_pred)

print("="*80)
print("MODEL PERFORMANCE COMPARISON:")
print("="*80)
print(f"\nTraining Set Performance:")
print(f"  RMSE: {train_rmse:.4f}")
print(f"  R²:   {train_r2:.4f}")

print(f"\nTest Set Performance:")
print(f"  RMSE: {model_rmse:.4f}")
print(f"  R²:   {model_r2:.4f}")

print(f"\nPerformance Gap:")
print(f"  RMSE difference: {abs(train_rmse - model_rmse):.4f}")
print(f"  R² difference:   {abs(train_r2 - model_r2):.4f}")

print(f"\n{'='*80}")
print("ANALYSIS:")
print(f"{'='*80}")

# Provide analysis
if model_r2 > 0.7:
    print("✓ Good model performance (R² > 0.7)")
elif model_r2 > 0.5:
    print("⚠ Moderate model performance (R² between 0.5 and 0.7)")
else:
    print("✗ Poor model performance (R² < 0.5)")

if abs(train_r2 - model_r2) < 0.1:
    print("✓ Similar train/test performance → No significant overfitting")
else:
    print("⚠ Gap between train/test performance → May indicate overfitting")

print(f"{'='*80}")

### Experiment with Different Learning Rates

Let's experiment with different learning rates to observe their effect on model convergence and performance.

In [ ]:
# Experiment with different learning rates
learning_rates = [0.000001, 0.00001, 0.0001, 0.001, 0.01]
iterations_exp = 1000

results = []

print("="*80)
print("EXPERIMENTING WITH DIFFERENT LEARNING RATES:")
print("="*80)

for alpha_exp in learning_rates:
    # Initialize weights
    W_init = np.zeros(X_train.shape[1])
    
    # Run gradient descent
    W_final, cost_hist = gradient_descent(X_train, Y_train, W_init, alpha_exp, iterations_exp)
    
    # Make predictions
    Y_pred_exp = np.dot(X_test, W_final)
    
    # Calculate metrics
    rmse_exp = rmse(Y_test, Y_pred_exp)
    r2_exp = r2(Y_test, Y_pred_exp)
    
    # Store results
    results.append({
        'learning_rate': alpha_exp,
        'final_cost': cost_hist[-1],
        'rmse': rmse_exp,
        'r2': r2_exp,
        'cost_history': cost_hist
    })
    
    print(f"\nLearning Rate: {alpha_exp:.6f}")
    print(f"  Initial Cost: {cost_hist[0]:.4f}")
    print(f"  Final Cost:   {cost_hist[-1]:.4f}")
    print(f"  RMSE:         {rmse_exp:.4f}")
    print(f"  R²:           {r2_exp:.4f}")

print(f"\n{'='*80}")

In [ ]:
# Visualize the effect of different learning rates
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, result in enumerate(results):
    ax = axes[idx]
    ax.plot(result['cost_history'], linewidth=2)
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Cost')
    ax.set_title(f"α = {result['learning_rate']:.6f}\nFinal R² = {result['r2']:.4f}")
    ax.grid(True, alpha=0.3)

# Hide the last subplot if we have an odd number
if len(results) < len(axes):
    axes[-1].axis('off')

plt.tight_layout()
plt.suptitle('Cost Function Convergence for Different Learning Rates', fontsize=16, y=1.02)
plt.show()

# Summary comparison
summary_df = pd.DataFrame([{
    'Learning Rate': r['learning_rate'],
    'Final Cost': f"{r['final_cost']:.4f}",
    'RMSE': f"{r['rmse']:.4f}",
    'R²': f"{r['r2']:.4f}"
} for r in results])

print("\n" + "="*80)
print("SUMMARY TABLE:")
print("="*80)
display(summary_df)